In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import time
from urllib.parse import quote
from PIL import Image

In [2]:
def capture_screenshot(url):
    options = Options()
    options.add_argument('--headless')  # Set the browser to run in headless mode

    # Initialize the Chrome driver
    driver = webdriver.Chrome(options=options)
    driver.get(url)
    print(f"Capturing screenshot for: {url}")

    # Set initial window size for full page capture
    initial_width = driver.execute_script("return Math.max(document.body.scrollWidth, document.body.offsetWidth, document.documentElement.clientWidth, document.documentElement.scrollWidth, document.documentElement.offsetWidth);")
    driver.set_window_size(initial_width, driver.get_window_size()['height'])

    # Scroll down the page to capture full content
    SCROLL_PAUSE_TIME = 2  # Adjust as needed
    last_height = driver.execute_script("return document.body.scrollHeight")
    while True:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(SCROLL_PAUSE_TIME)
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height
    # Take individual screenshots and save them
    total_height = driver.execute_script("return document.body.scrollHeight")
    driver.set_window_size(initial_width, total_height)
    
    scroll_position = 0
    screenshot_counter = 0
    while scroll_position < total_height:
        driver.execute_script(f"window.scrollTo(0, {scroll_position});")
        driver.save_screenshot(f"images/screenshot_{screenshot_counter}.png")
        screenshot_counter += 1
        scroll_position += driver.get_window_size()['height']

    imgs = [Image.open(f'images/screenshot_{i}.png') for i in range(screenshot_counter)]
    imgs_comb = Image.new('RGB', (imgs[0].width, sum(img.height for img in imgs)))
    y = 0
    for img in imgs:
        imgs_comb.paste(img, (0, y))
        y += img.height
    filename = quote(url, safe='').replace('/', '_')
    imgs_comb.save(f'images/{filename}.png')
    #print('Full screenshot captured')
    print(f"Screenshot saved for: {url}")


In [3]:
URL = [
    'https://www.calliopee.ch/domiciliation',
    'https://www.calliopee.ch/business-center',
    # Add more URLs as needed
]

In [4]:
for url in URL:
    capture_screenshot(url)

Capturing screenshot for: https://www.calliopee.ch/domiciliation
Screenshot saved for: https://www.calliopee.ch/domiciliation
Capturing screenshot for: https://www.calliopee.ch/business-center
Screenshot saved for: https://www.calliopee.ch/business-center
